<div align="center">
<h2>Building Data Ingestion from Scratch</h2>
</div>

<div align="center">
    <h3><a href="https://aiengineering.academy/" target="_blank">AI Engineering.academy</a></h3>
</div>

<div align="center">
<a href="https://aiengineering.academy/" target="_blank">
<img src="https://raw.githubusercontent.com/adithya-s-k/AI-Engineering.academy/main/assets/banner.png" alt="AI Engineering Academy" width="50%">
</a>
</div>

<div align="center">

[![GitHub Stars](https://img.shields.io/github/stars/adithya-s-k/AI-Engineering.academy?style=social)](https://github.com/adithya-s-k/AI-Engineering.academy/stargazers)
[![GitHub Forks](https://img.shields.io/github/forks/adithya-s-k/AI-Engineering.academy?style=social)](https://github.com/adithya-s-k/AI-Engineering.academy/network/members)
[![GitHub Issues](https://img.shields.io/github/issues/adithya-s-k/AI-Engineering.academy)](https://github.com/adithya-s-k/AI-Engineering.academy/issues)
[![GitHub Pull Requests](https://img.shields.io/github/issues-pr/adithya-s-k/AI-Engineering.academy)](https://github.com/adithya-s-k/AI-Engineering.academy/pulls)
[![License](https://img.shields.io/github/license/adithya-s-k/AI-Engineering.academy)](https://github.com/adithya-s-k/AI-Engineering.academy/blob/main/LICENSE)

</div>


## Introduction

Data ingestion is a crucial first step in building effective Retrieval-Augmented Generation (RAG) systems. It involves the process of collecting, processing, and storing data in a format that can be efficiently retrieved and used by the RAG model. This README provides an overview of the data ingestion process for RAG systems.

## Importance of Data Ingestion in RAG

Effective data ingestion is essential for RAG systems because it:
1. Determines the quality and relevance of information available for retrieval.
2. Affects the system's ability to understand and process queries accurately.
3. Impacts the overall performance and efficiency of the RAG pipeline.
4. Enables the system to handle diverse data sources and formats.

## Key Steps in Data Ingestion

The data ingestion process typically involves the following steps:

```mermaid
flowchart TB
    A[Data Collection] --> B[Data Cleaning]
    B --> C[Document Splitting]
    C --> D[Metadata Extraction]
    D --> E[Embedding Generation]
    E --> F[Indexing and Storage]
```

1. **Data Collection**: Gathering information from various sources such as databases, APIs, web scraping, or file systems.

2. **Data Cleaning**: Preprocessing the collected data to remove noise, handle missing values, and standardize formats.

3. **Document Splitting**: Breaking down large documents into smaller, manageable chunks for more effective retrieval.

4. **Metadata Extraction**: Identifying and extracting relevant metadata from the documents to enhance retrieval capabilities.

5. **Embedding Generation**: Creating vector representations of the text chunks to enable semantic search.

6. **Indexing and Storage**: Organizing and storing the processed data in a format optimized for quick retrieval, often using vector databases or search engines.

## Challenges in Data Ingestion

- Handling diverse data formats and sources
- Ensuring data quality and consistency
- Managing large volumes of data efficiently
- Updating and maintaining the knowledge base
- Balancing between chunk size and semantic coherence

## Best Practices

1. **Data Quality**: Implement robust data cleaning and validation processes.
2. **Scalability**: Design the ingestion pipeline to handle growing data volumes.
3. **Metadata Enrichment**: Extract and store relevant metadata to improve retrieval accuracy.
4. **Incremental Updates**: Develop mechanisms for efficiently updating the knowledge base.
5. **Monitoring**: Implement logging and monitoring to track ingestion performance and data quality.

## Conclusion

A well-designed data ingestion process is fundamental to the success of a RAG system. It ensures that the information retrieved is accurate, relevant, and up-to-date, ultimately leading to better-quality responses from the language model.

In the following sections, we'll explore other crucial components of RAG systems, including data chunking, embedding generation, and retrieval mechanisms.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!!pip install llama-index
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai
!pip install llama-index-vector-stores-qdrant
!pip -q install python-dotenv 
!pip install -U qdrant_client fastembed
!pip install pymupdf

## OpenAI

You will need an [OpenAI](https://openai.com/) api key for this tutorial. Login to your [platform.openai.com](https://platform.openai.com/) account, click on your profile picture in the upper right corner, and choose 'API Keys' from the menu. Create an API key for this tutorial and save it. You will need it below.

Set your OpenAI api key, and Pinecone api key and environment in the file we created.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

## Setting up Vector Database

We will be using qDrant as the Vector database
There are 4 ways to initialize qdrant 

1. Inmemory
```python
client = qdrant_client.QdrantClient(location=":memory:")
```
2. Disk
```python
client = qdrant_client.QdrantClient(path="./data")
```
3. Self hosted or Docker
```python

client = qdrant_client.QdrantClient(
    # url="http://<host>:<port>"
    host="localhost",port=6333
)
```

4. Qdrant cloud
```python
client = qdrant_client.QdrantClient(
    url=QDRANT_CLOUD_ENDPOINT,
    api_key=QDRANT_API_KEY,
)
```

for this notebook we will be using qdrant cloud

In [ ]:
import qdrant_client

# LlamaIndex core imports
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

# LlamaIndex vector store import
from llama_index.vector_stores.qdrant import QdrantVectorStore

# creating a qdrant client instance

client = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    # location=":memory:"
    # otherwise set Qdrant instance address with:
    # url=QDRANT_CLOUD_ENDPOINT,
    # otherwise set Qdrant instance with host and port:
    host="localhost",
    port=6333
    # set API KEY for Qdrant Cloud
    # api_key=QDRANT_API_KEY,
    # path="./db/"
)

vector_store = QdrantVectorStore(client=client, collection_name="01_Data_Ingestion")

## Build an Ingestion Pipeline from Scratch

We show how to build an ingestion pipeline as mentioned in the introduction.

Note that steps (2) and (3) can be handled via our `NodeParser` abstractions, which handle splitting and node creation.

For the purposes of this tutorial, we show you how to create these objects manually.

### 1. Load Data

In [ ]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "../data/llama2.pdf"

--2023-10-13 01:45:14--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 128.84.21.199
Connecting to arxiv.org (arxiv.org)|128.84.21.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  7.59MB/s    in 1.7s    

2023-10-13 01:45:16 (7.59 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]


In [ ]:
import fitz

In [ ]:
file_path = "../data/llama2.pdf"
doc = fitz.open(file_path)

### 2. Use a Text Splitter to Split Documents

Here we import our `SentenceSplitter` to split document texts into smaller chunks, while preserving paragraphs/sentences as much as possible.

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

In [ ]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [ ]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, page in enumerate(doc):
    page_text = page.get_text("text")
    cur_text_chunks = text_parser.split_text(page_text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

### 3. Manually Construct Nodes from Text Chunks

We convert each chunk into a `TextNode` object, a low-level data abstraction in LlamaIndex that stores content but also allows defining metadata + relationships with other Nodes.

We inject metadata from the document into each node.

This essentially replicates logic in our `SentenceSplitter`.

In [ ]:
from llama_index.core.schema import TextNode

In [ ]:
nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc_idx = doc_idxs[idx]
    src_page = doc[src_doc_idx]
    nodes.append(node)

In [ ]:
print(nodes[0].metadata)

In [ ]:
# print a sample node
print(nodes[0].get_content(metadata_mode="all"))

### [Optional] 4. Extract Metadata from each Node

We extract metadata from each Node using our Metadata extractors.

This will add more metadata to each Node.

In [ ]:
from llama_index.core.extractors import (
    QuestionsAnsweredExtractor,
    TitleExtractor,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, llm=llm),
]

In [ ]:
pipeline = IngestionPipeline(
    transformations=extractors,
)
nodes = await pipeline.arun(nodes=nodes, in_place=False)

In [ ]:
print(nodes[0].metadata)

### 5. Generate Embeddings for each Node

Generate document embeddings for each Node using our OpenAI embedding model (`text-embedding-ada-002`).

Store these on the `embedding` property on each Node.

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()

In [ ]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

### 6. Load Nodes into a Vector Store

We now insert these nodes into our `PineconeVectorStore`.

**NOTE**: We skip the VectorStoreIndex abstraction, which is a higher-level abstraction that handles ingestion as well. We use `VectorStoreIndex` in the next section to fast-track retrieval/querying.

In [ ]:
vector_store.add(nodes)

## Retrieve and Query from the Vector Store

Now that our ingestion is complete, we can retrieve/query this vector store.

**NOTE**: We can use our high-level `VectorStoreIndex` abstraction here. See the next section to see how to define retrieval at a lower-level!

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

In [ ]:
index = VectorStoreIndex.from_vector_store(vector_store)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query_str = "Can you tell me about the key concepts for safety finetuning"

In [ ]:
response = query_engine.query(query_str)

In [ ]:
print(str(response))